In [1]:
import json

import altair
import intake
import ipyleaflet
import ipywidgets
import pandas
import geopandas
from IPython.display import Markdown

cat = intake.open_catalog("../catalogs/*.yml")

In [2]:
city = cat.la_geohub.city_boundary.read()
county = cat.la_geohub.county_boundary.read()
cpa = cat.la_geohub.community_plan_area.read()

In [3]:
pep = cat.pep_geocode.read()

In [4]:
unlocatable = [
    "confidental",
    "confidential",
    "various",
    "citywide",
    "city-wide",
    "n/a",
    "see attached"
]
unlocatable_pep = pep[
    pep.pep_proj_loc_addr.str.lower().str.contains('|'.join(unlocatable)) |
    (pep.citywide_cncl_dist_yn == 1)
]

located_pep = pep[
    ~pep.pep_proj_loc_addr.str.lower().str.contains('|'.join(unlocatable)) &
    ~(pep.citywide_cncl_dist_yn == 1)
]

In [5]:
pep_in_city = located_pep.within(city.iloc[0].geometry)
unlocated_pep = located_pep[~pep_in_city]
located_pep = located_pep[pep_in_city]

In [6]:
pep_cpa = geopandas.sjoin(
    located_pep,
    cpa[["NAME", "geometry"]],
    how="left",
    op="within",
)

In [7]:
cpa_agg = pep_cpa.groupby(["NAME", "year"]).agg({
    "cdbg_fnd_amt": lambda x: sum(x)/1.e6
}).reset_index(level=1).rename(columns={"cdbg_fnd_amt": "amount"})

In [8]:
# Create the base map
m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.CartoDB.Positron)

# Create the choropleth layer
def get_choro_data(year):
    df = cpa.set_index("NAME").assign(
        amount=cpa_agg[cpa_agg.year == year].amount
    ).fillna(0)[
        ["geometry", "amount"]
    ]
    choro_data = df.amount.to_dict()
    geo_data = json.loads(df.to_json())
    return choro_data, geo_data

init_year = 44
data = get_choro_data(init_year)

choro_layer = ipyleaflet.Choropleth(
    style={'fillOpacity': 0.6},
    geo_data=data[1],
    choro_data=data[0],
    vmin=0.0,
    vmax=cpa_agg.amount.max()
)
m.add_layer(choro_layer)

# Create the individual project layer
circle_layer = ipyleaflet.LayerGroup(markers=[])

def update_circles(year):
    circles = []
    projects = pep_cpa[pep_cpa.year == year]
    for idx, row in projects.iterrows():
        circles.append(ipyleaflet.Circle(
            location=(row.geometry.y, row.geometry.x),
            radius=int(row.cdbg_fnd_amt/5.e3),
            weight=1,
            popup=ipywidgets.HTML(value=f"""
            <b>Project: </b>{row.pep_proj_nm} <br>
            <b>Agency: </b>{row.pep_agcy_nm} <br>
            <b>Address: </b>{row.address} <br>
            <b>Fund amount </b>${row.cdbg_fnd_amt:,.0f} <br>
            """),
        ))
    circle_layer.layers=circles
m.add_layer(circle_layer)

# Callback to update the map data when the slider is changed
def update_map(year):
    data = get_choro_data(year)
    choro_layer.geo_data = data[1]
    choro_layer.choro_data = data[0]
    choro_layer.vmin=0.0,
    choro_layer.vmax=cpa_agg.amount.max()
    update_circles(year)

# Create the slider
slider = ipywidgets.IntSlider(
    description="Grant Year",
    min=cpa_agg.year.min(),
    max=cpa_agg.year.max(),
    value=init_year,
    continuous_update=False
)

def on_selection(*args):
    update_map(slider.value)
slider.observe(on_selection, names=["value"])
m.add_control(ipyleaflet.WidgetControl(widget=slider, position="topright"))

# Create the CPA label
cpa_label = ipywidgets.HTML(value="<b>Community Plan Area: </b><i>Hover to select</i>")

def on_hover(**kwargs):
    properties = kwargs.get("feature", {}).get("properties")
    id = kwargs.get("feature", {}).get("id")
    if not properties:
        return
    cpa_label.value=f"""
    <b>Community Plan Area: </b> {id} <br>
    <b>Total funding: </b> ${properties['amount']:,.1f}M
    """
choro_layer.on_hover(on_hover)
m.add_control(ipyleaflet.WidgetControl(widget=cpa_label, position="topright"))

# Finialize the map.
update_map(init_year)
m.center = [34.07996230865876, -118.31123326410754]
m.zoom = 10

# Show the map
m

Map(center=[34.07996230865876, -118.31123326410754], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [9]:
confidential_pep = pep[
    pep.pep_proj_loc_addr.str.lower().str.contains("confidental|confidential") &
    (pep.citywide_cncl_dist_yn == 1)
].groupby("year").agg({"cdbg_fnd_amt": lambda x: sum(x)/1.e6}).reset_index().assign(
    NAME="Confidential"
).rename(columns={"cdbg_fnd_amt": "amount"})
citywide_pep = pep[
    pep.pep_proj_loc_addr.str.lower().str.contains("citywide|city-wide") &
    (pep.citywide_cncl_dist_yn == 1)
].groupby("year").agg({"cdbg_fnd_amt": lambda x: sum(x)/1.e6}).reset_index().assign(
    NAME="Citywide"
).rename(columns={"cdbg_fnd_amt": "amount"})

In [10]:
to_chart = pandas.concat([cpa_agg.reset_index(), confidential_pep, citywide_pep])

In [11]:
hbox = ipywidgets.HBox()
o1 = ipywidgets.Output()
o2 = ipywidgets.Output()

with o1:
    areas = to_chart.groupby("NAME").amount.sum().sort_values().tail(10).index
    display(
        altair.Chart(
            to_chart[to_chart.NAME.isin(areas)]
        ).mark_line(strokeWidth=4).encode(
            x=altair.X("year", title="Grant year"),
            y=altair.Y("amount", title="Funding amount (million $)"),
            color=altair.Color('NAME', legend=altair.Legend(
                direction="vertical",
                orient="top",
                title="Region",
            )),
            tooltip=['NAME', 'amount', 'year'],
        )
    )
with o2:
    display(
        to_chart.rename(
            columns={"NAME": "Region"}
        ).groupby("Region").agg({"amount": "sum"}).sort_values("amount", ascending=False).rename(
            columns={"amount": "Funding amount (since grant year 32)"}
        ).style.format(lambda x: f"${x:,.1f} million")
    )
    
hbox.children = [o1, o2]
hbox

In [12]:
display(Markdown(
    f"Number of confidential, city-wide, or otherwise unlocatable PEPs: {len(unlocatable_pep)}"
))
display(Markdown(
    f"Number of unlocated PEPs, or ones outside of the City of LA: {len(unlocated_pep)}"
))
display(Markdown(
    f"Number of located PEPs: {len(located_pep)}"
))


Number of confidential, city-wide, or otherwise unlocatable PEPs: 934

Number of unlocated PEPs, or ones outside of the City of LA: 126

Number of located PEPs: 1803